<a href="https://colab.research.google.com/github/Georgia-MAX-holic/theory/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### XGBoost ( eXtra Gradient Boost ) 

1. 주요 장점 
  - 뛰어난 예측 성능 

  - GBM 대비 빠른 수행 시간 ( CPU병렬 처리 , GPU 지원)

  - 다양한 성능 향상 기능 

     - 규제(Regularization) 기능 탑재 
     
     - Tree prunding 

  - 다양한 편의 기능 
     - 조기 중단 ( Early Stopping) 

     - 자체 내장된 교차 검증 

     - 결손값 자체 처리 





---
XGBoost 조기 중단 기능 (Early Stopping) 

- XGBoost 는 특정 반복 횟수 만큼 더 이상 비용함수가 감소하지 않으면 지정된 반복횟수를 다 완료하지 않고 수행을 종료할 수 있음 

- 학습을 위한 시간을 단축 시킬 수 있음. 특히 최적화 튜닝 단계}에서 적절하게 사용가능 

- 너무 반복 횟수를 단축할 경우 예측 성능 최적화가 안된 상태에서 학습이 종료될 수 있으므로 유의 필요 

- 조기 중단 설정을 위한 주요 파라미터 

   - early_stopping_rounds




---
# **LightGBM 개요**

XGBoost 대비 장점 

- 더 빠른 학습과 예측 수행 시간

- 더 작은 메모리 사용량 

- 카테고리형 피처의 자동 변환과 최적 분할( 원 - 핫 인코딩 ) 
등을 사용하지 않고도 카테고리형 피처를 최적으로 변환하고 이에 따른 노드 분할 수행 가능 


LightGBM 트리 분할 방식 - 리프 중심 

- 일반적은 GBM 계열의 트리 분할 방법은 트리의 깊이를 효과적으로 줄이기 위해 " 균형 분할 방식"을 사용 (오버 피팅에 가함 ) 

- 최대 손실 값을 가지는 리프 노드를 지속적으로 분할하면서 트리의 깊이가 깊어지고 비대칭적인 규칙 트리가 생성되는것이 특징 




---


# **사이킷런 래퍼 LightGBM 하이퍼파라미터** 

- n_estimators 
   - 반복을 수행하려는 트리의 개수 

   - 기본값은 100

- learning_rate 
   - 기본값은 

- max_depth
   - Depth wise 방식이 아닌 Leaf Wise 방식이므로 깊이가 상대적으로 더 깊다는 점 

   - 0보다 작은 값을 지정하면 깊이에 제한이 없음 

   - 기본값은 -1 


- num_leaves 
   - 하나의 트리가 가질 수 있는 최대 리프 개수 
   - 기본값은 31 



---

# **LightGBM 과적합 감소 방법**

모델 복잡도를 제어하는 주요 파라미터 

-  num_leaves 

- min_child_samples

- max_depth 

In [20]:
from lightgbm import LGBMClassifier
import lightgbm
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
dataset = load_breast_cancer()

In [3]:
cancer_df = pd.DataFrame(data=dataset, columns = dataset.feature_names)

In [4]:
cancer_df["target"] = dataset.target 

In [22]:
## data split => train / test
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2, random_state=156)

## data split => train => train / val
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=156)

In [23]:
lgbm_wrapper = LGBMClassifier(n_estimators=400, learning_rate=0.05)

In [24]:
evals = [(X_tr, y_tr), (X_val, y_val)]

In [ ]:
lgbm_wrapper.fit(X_tr, y_tr,
                 early_stopping_rounds=50,
                 eval_metric="logloss",
                 eval_set=evals,
                 verbose=True
                 )

In [ ]:
#모델 평가

preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)

## 모델 성능 평가 함수 선언 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):

    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)

    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC: {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


In [ ]:
get_clf_eval(y_test, preds, pred_proba)